In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True) #mounting my google drive

In [ ]:
!mkdir ~/.kaggle

In [28]:
!cp /content/drive/MyDrive/KAGGLE_API_CREDENTIALS/kaggle.json ~/.kaggle/kaggle.json

In [29]:
!chmod 600 ~/.kaggle/kaggle.json

In [30]:
import torch
from torch.optim import Adam
import matplotlib.pyplot as plt
from torch import nn
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report
import copy

In [31]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' #setting up device agnostic code

In [ ]:
!kaggle datasets download -d ananthu017/emotion-detection-fer #downloading dataset in zip form

In [ ]:
!unzip /content/emotion-detection-fer.zip #unzipping dataset

In [131]:
train_transform = transforms.Compose([ #transfromations i want applied on train images
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Grayscale(num_output_channels=1),  # convert to grayscale
    transforms.Normalize(mean=[0.485], std=[0.229])
])

test_transform = transforms.Compose([ #transfromations i want applied on test images
    transforms.Grayscale(num_output_channels=1),  # convert to grayscale
    transforms.ToTensor()
])

In [132]:
train_dataset = datasets.ImageFolder('/content/train', transform=train_transform)
test_dataset = datasets.ImageFolder('/content/test', transform=test_transform)

In [133]:
#it's an imbalanced dataset, so I'm taking into account the number of each classes

angry_label = train_dataset.class_to_idx['angry']  # get the label of 'angry'
num_angry_images = sum(label == angry_label for _, label in train_dataset)  # count how many images have this label

disgusted_label = train_dataset.class_to_idx['disgusted']  # get the label of 'disgusted'
num_disgusted_images = sum(label == disgusted_label for _, label in train_dataset)

fearful_label = train_dataset.class_to_idx['fearful']  # get the label of 'fearful'
num_fearful_images = sum(label == fearful_label for _, label in train_dataset)

happy_label = train_dataset.class_to_idx['happy']  # get the label of 'happy'
num_happy_images = sum(label == happy_label for _, label in train_dataset)

neutral_label = train_dataset.class_to_idx['neutral']  # get the label of 'neutral'
num_neutral_images = sum(label == neutral_label for _, label in train_dataset)

sad_label = train_dataset.class_to_idx['sad']  # get the label of 'sad'
num_sad_images = sum(label == sad_label for _, label in train_dataset)

surprised_label = train_dataset.class_to_idx['surprised']  # get the label of 'surprised'
num_surprised_images = sum(label == surprised_label for _, label in train_dataset)



In [134]:
total_images = len(train_dataset)

weights = [
    total_images / num_angry_images,
    total_images / num_disgusted_images,
    total_images / num_fearful_images,
    total_images / num_happy_images,
    total_images / num_neutral_images,
    total_images / num_sad_images,
    total_images / num_surprised_images,
]

weights = [weight / sum(weights) for weight in weights] #normalizing the weights so they add up to 1


In [135]:
print(weights)

[0.06857175062953616, 0.6283122563417362, 0.06686457011593774, 0.03796869629452487, 0.055175054131922856, 0.05671721403002007, 0.08639045845632197]


In [136]:
BATCH_SIZE = 32 #splitting my datasets into random mini batches

train_loader = DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_dataset,batch_size=BATCH_SIZE, shuffle=True)

In [137]:

class Emotion_Detection_Model_V1(nn.Module):
  def __init__(self, input_shape , output_shape):
    super().__init__()

    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape, out_channels=96, stride=1, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels = 96, out_channels=108, stride=1, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )

    self.conv_block_2 = nn.Sequential(
        nn.Conv2d(in_channels=108, out_channels=120, stride=1, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels = 120, out_channels=140, stride=1, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )

    self.conv_block_3 = nn.Sequential(
        nn.Conv2d(in_channels=140, out_channels=160, stride=1, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels = 160, out_channels=200, stride=1, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )

    self.conv_block_4 = nn.Sequential(
        nn.Conv2d(in_channels=200, out_channels=260, stride=1, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels = 260, out_channels=300, stride=1, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )

    self.classifier_layer = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=2700,out_features=output_shape)
    )

  def forward(self,X):
    X = self.conv_block_1(X)
    X = self.conv_block_2(X)
    X = self.conv_block_3(X)
    X = self.conv_block_4(X)
    X = self.classifier_layer(X)

    return X


In [138]:
model_0 = Emotion_Detection_Model_V1(input_shape=1,output_shape=len(train_dataset.classes)).to(device)

In [139]:
weights = torch.Tensor(weights).to(device)
loss_fn = nn.CrossEntropyLoss(weight=weights)


In [140]:

def train_stage(model=None,loss_fn=None, learning_rate=None,train_loader=None):


  optimizer = Adam(params=model.parameters(), lr=learning_rate)
  model.train()

  for X_train,y_train in train_loader:
    X_train,y_train = X_train.to(device), y_train.to(device)
    optimizer.zero_grad()
    y_pred = model(X_train)
    train_loss = loss_fn(y_pred,y_train)
    train_loss.backward()
    optimizer.step()



In [141]:
def test_stage(model=None,test_loader=None):
  with torch.inference_mode():
    model.eval()
    y_trues = []
    y_preds = []
    for X_test,y_test in test_loader:
      X_test, y_test = X_test.to(device), y_test.to(device)
      y_pred = model(X_test)

      y_trues.extend(list(y_test.cpu().numpy()))
      y_preds.extend(list(y_pred.argmax(dim=1).cpu().numpy()))

    return (y_trues,y_preds)

In [ ]:
epochs = 100
best_accuracy = 0.0
best_report = None
best_model_weights = model_0.state_dict()

for epoch in range(epochs):
  train_stage(model=model_0,loss_fn=loss_fn,learning_rate=0.01,train_loader=train_loader)

  if epoch % 5 == 0:
    y_trues, y_preds = test_stage(model=model_0, test_loader=test_loader)
    report = classification_report(
        y_true=y_trues,
        y_pred=y_preds,
        output_dict=True,
        target_names=test_dataset.classes,
        zero_division='warn'
    )

    if report['accuracy'] > best_accuracy:
      best_report = report
      best_model_weights = copy.deepcopy(model_0.state_dict())

  print(epoch)



In [143]:
print(best_report)

{'angry': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 958}, 'disgusted': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 111}, 'fearful': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1024}, 'happy': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1774}, 'neutral': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1233}, 'sad': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1247}, 'surprised': {'precision': 0.11577040958484257, 'recall': 1.0, 'f1-score': 0.20751654388812585, 'support': 831}, 'accuracy': 0.11577040958484257, 'macro avg': {'precision': 0.016538629940691794, 'recall': 0.14285714285714285, 'f1-score': 0.02964522055544655, 'support': 7178}, 'weighted avg': {'precision': 0.01340278773544221, 'recall': 0.11577040958484257, 'f1-score': 0.02402427528155929, 'support': 7178}}
